<a href="https://colab.research.google.com/github/thesteve0/impatient-computer-vision/blob/main/6_keypoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keypoint Detection

Alright, now we are going to do something a bit more specialized and, I think, amazing - keypoint detection. With this technique, the model detects key points in the objects and identifies or tracks them. An example of keypoints is landmarks on the human face such as eyes, ears, nose, mouth, chin.... While we will only be doing this on still 2D images, you can doing this on 3D images, and on movies as well. Once you have identitied keypoints there is all sorts of downstream analysis you can do such, pose estimation, emotions, attention direction, and many other use cases. This tecnhique can be used beyond humans with it being used for animals, or any key identifying objects you want to track.

Let's set up the house:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install fiftyone==1.4.1 torch torchvision ultralytics

import fiftyone as fo
from fiftyone import ViewField as F
import fiftyone.utils.ultralytics as fou
from ultralytics.engine.results import Keypoints
from ultralytics import YOLO


name = "our-photos"
dir = "/content/drive/MyDrive/impatient-cv/flickr-labeled"

dataset = fo.Dataset.from_dir(
    dataset_dir=dir,
    dataset_type=fo.types.FiftyOneDataset,
    name=name
)


## Download and running the model

Setting up a keypoint model is a little more involved if you want to leverage all its value. Out of the box, the model will identify points it has been trained to detect. But quite often, you want to label those points, and then you want to connect points to form a "skeleton."

This time, instead of downloading a model from the Zoo, we will use one of the [Yolo models](https://docs.ultralytics.com/tasks/pose/#models) provided by Ultralytics. They come in many different sizes depending on your needs for speed versus accuracy, we will go with the middle of the road - `YOLO11m-pose`. FiftyOne comes with an [out-of-the-box integration](https://docs.voxel51.com/integrations/ultralytics.html) with Ultralytics, which greatly reduces the overhead of bringing in the model.


In [ ]:

model = YOLO("yolo11m-pose.pt")

# We are only going to do keypoint detection for those images where the ground truth says there are people in the image
people_view = dataset.filter_labels("ground_truth", (F("label") == "people"))

#Because of a bug in the default code we need to iterate through each sample in the dataset, apply the model, convert the keypoints, and save the sample - https://github.com/voxel51/fiftyone/issues/5896
for sample in people_view.iter_samples(progress=True):
    # We are applying the model one image at a time
    result = model(sample.filepath)[0]

    # Workaround for bug - need to clone because PyTorch tensors are not mutable in inference mode
    xy_clone = result.keypoints.xy.clone()
    # Replace zeros with NaN
    xy_clone[xy_clone == 0.0000] = float('nan')
    new_keypoints = Keypoints(xy_clone, result.orig_shape)
    result.keypoints = new_keypoints
    sample["new_keypoints"] = fou.to_keypoints(result)

    sample.save()

# Code that is not working properly right now - uncomment if you want to see
# dataset.apply_model(model, label_field="broken_keypoints")

# dataset.default_skeleton = fo.KeypointSkeleton(
#     labels=[
#         "nose", "left eye", "right eye", "left ear", "right ear",
#         "left shoulder", "right shoulder", "left elbow", "right elbow",
#         "left wrist", "right wrist", "left hip", "right hip",
#         "left knee", "right knee", "left ankle", "right ankle",
#     ],
#     edges=[
#         [11, 5, 3, 1, 0, 2, 4, 6, 12],
#         [9, 7, 5, 6, 8, 10],
#         [15, 13, 11, 12, 14, 16],
#     ],
# )

session = fo.launch_app(dataset, auto=False)
session.url




## Wrap up

From here you can do all sorts of analysis to ask questions of the picture. For example, the slope of the line between the head and the hips could tell you if the person is laying down, leaning, or standing straight. The angle at the elbow detection could tell if if the arms is bent and by how much.

We are getting towards the end of the workshop. The next notebook will cover one of the areas receiving a lot of attention right now in computer vision - visual language models. The final topic, fine tuning a model, will have a notebook but it won't be run in class. Instead, the notebook will just serve for you to examine for future reference.

[Visual Language Models](https://github.com/thesteve0/impatient-computer-vision/blob/main/7_visual_language_models.ipynb)